In [42]:
import os
import sqlite3
import pandas as pd

In [76]:
def creer_table_si_absente(cursor, nom_table, colonnes,dtype):
    """
    Crée une table dans la base de données si elle n'existe pas déjà,
    en utilisant les colonnes extraites de l'en-tête du fichier CSV.
    """
    colonnes_avec_types = []
    for colonne in colonnes:
        type_colonne = dtype.get(colonne, 'TEXT')  # Utiliser 'TEXT' par défaut si le type n'est pas spécifié
        if type_colonne == 'float':
            type_colonne = 'REAL'
        elif type_colonne == 'int':
            type_colonne = 'INTEGER'
        colonnes_avec_types.append(f"{colonne} {type_colonne}")
    colonnes_avec_types_str = ', '.join(colonnes_avec_types)
    requete_creation = f"CREATE TABLE IF NOT EXISTS {nom_table} ({colonnes_avec_types_str});"
    print(requete_creation)
    cursor.execute(requete_creation)


In [77]:
def inserer_donnees(cursor, nom_table, colonnes, donnees):
    """
    Insère les données dans la table. 
    """
    placeholders = ', '.join(['?' for _ in colonnes])
    colonnes_str = ', '.join(colonnes)
    requete_insertion = f"INSERT INTO {nom_table} ({colonnes_str}) VALUES ({placeholders});"
    cursor.executemany(requete_insertion, donnees)

In [78]:
def extraire_info_du_nom_fichier(fichier,liste_nom_table):  
    # extraction informations du nom de fichier à encoder dans la table
    annee_mois=fichier[:7]
    info_geo=fichier[8:-4]
    # where sans l'élément qui est dans la liste
    for i in liste_nom_table:
        if i in info_geo:
            info_geo=info_geo.replace(i,"")[:-1]
    return annee_mois,info_geo

In [79]:
def nettoyer_noms_colonnes(colonnes):
    """
    Nettoie les noms des colonnes pour éviter les problèmes de requêtes SQL.
    """
     # nettoyage des noms de colonnes
    colonnes = [colonne.replace(' ', '') for colonne in colonnes]
    colonnes = [colonne.replace('-', '') for colonne in colonnes]
    colonnes = [colonne.replace('_', '') for colonne in colonnes]
    colonnes_nettoyees = [colonne.replace(' ', '').replace('-', '').replace('_', '') for colonne in colonnes]
    return colonnes_nettoyees

In [80]:
def nettoyer_nom_table(liste_nom_table,chemin_fichier):
    for nom_table in liste_nom_table:
        if nom_table in chemin_fichier:
            nom_table = nom_table.replace('-', '')
            break
        else:
            nom_table = 'autre'
            
    print(f"nom de la table : {nom_table}")
    # Créer la table si elle n'existe pas
    return nom_table


In [81]:
def traiter_fichier(chemin_fichier, fichier, connection,liste_nom_table,dtype):
    """
    Traite un fichier CSV en créant une table correspondante et en y insérant les données.
    """
    # Gérer les valeurs manquantes
    na_values = ['NA', 'N/A', '']

    df = pd.read_csv(chemin_fichier, dtype=dtype, na_values=na_values)

    # rajout des informations year_month et where dans le dataframe
    annee_mois,info_geo = extraire_info_du_nom_fichier(fichier,liste_nom_table)
    df['annee_mois'] = annee_mois
    df['info_geo'] = info_geo
    
    nom_table = nettoyer_nom_table(liste_nom_table,chemin_fichier)
    colonnes = nettoyer_noms_colonnes(list(df.columns))

    # Insérer les données dans la table
    cursor = connection.cursor()
    creer_table_si_absente(cursor, nom_table, colonnes,dtype)
    inserer_donnees(cursor, nom_table, colonnes, df.values.tolist())

    # Commit les changements
    connection.commit()

    print(f"Traitement du fichier : {chemin_fichier} terminé.")

    return df

In [82]:
def parcourir_arborescence(chemin_racine, db_path,liste_nom_table,dtype):
    """
    Parcourt récursivement l'arborescence et traite chaque fichier CSV trouvé.
    """
    # Créer ou se connecter à la base de données
    connection = sqlite3.connect(db_path)

    for racine, sous_repertoires, fichiers in os.walk(chemin_racine):
         for fichier in fichiers:
            if fichier.endswith(".csv"):
                chemin_fichier = os.path.join(racine, fichier)
                df=traiter_fichier(chemin_fichier, fichier,connection,liste_nom_table,dtype)
    # Fermer la connexion à la base de données
    connection.close()
    return df

In [83]:

chemin_racine = "C:/Users/Admin.local/Documents/projetint/files_test"

db_path = "data.db" 

# on définit le nom des tables en fonction du nom du fichier (terminaison)
liste_nom_table = ['outcomes','stop-and-search','street']

dtype={'Longitude': 'float', 'Latitude': 'float'}

df=parcourir_arborescence(chemin_racine, db_path,liste_nom_table,dtype)


nom de la table : outcomes
CREATE TABLE IF NOT EXISTS outcomes (CrimeID TEXT, Month TEXT, Reportedby TEXT, Fallswithin TEXT, Longitude REAL, Latitude REAL, Location TEXT, LSOAcode TEXT, LSOAname TEXT, Outcometype TEXT, anneemois TEXT, infogeo TEXT);
Traitement du fichier : C:/Users/Admin.local/Documents/projetint/files_test\files\2021-10-avon-and-somerset-outcomes.csv terminé.
nom de la table : stopandsearch
CREATE TABLE IF NOT EXISTS stopandsearch (Type TEXT, Date TEXT, Partofapolicingoperation TEXT, Policingoperation TEXT, Latitude REAL, Longitude REAL, Gender TEXT, Agerange TEXT, Selfdefinedethnicity TEXT, Officerdefinedethnicity TEXT, Legislation TEXT, Objectofsearch TEXT, Outcome TEXT, Outcomelinkedtoobjectofsearch TEXT, Removalofmorethanjustouterclothing TEXT, anneemois TEXT, infogeo TEXT);
Traitement du fichier : C:/Users/Admin.local/Documents/projetint/files_test\files\2021-10-avon-and-somerset-stop-and-search.csv terminé.
nom de la table : street
CREATE TABLE IF NOT EXISTS str

In [86]:
df.head(2)

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,annee_mois,info_geo
0,NaN,2021-10,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.494870,51.422276,On or near Conference/Exhibition Centre,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN,2021-10,avon-and-somerset
1,NaN,2021-10,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.511761,51.409966,On or near Caernarvon Close,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN,2021-10,avon-and-somerset


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14554 entries, 0 to 14553
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Crime ID               12241 non-null  object 
 1   Month                  14554 non-null  object 
 2   Reported by            14554 non-null  object 
 3   Falls within           14554 non-null  object 
 4   Longitude              13411 non-null  float64
 5   Latitude               13411 non-null  float64
 6   Location               14554 non-null  object 
 7   LSOA code              13411 non-null  object 
 8   LSOA name              13411 non-null  object 
 9   Crime type             14554 non-null  object 
 10  Last outcome category  12241 non-null  object 
 11  Context                0 non-null      float64
 12  annee_mois             14554 non-null  object 
 13  info_geo               14554 non-null  object 
dtypes: float64(3), object(11)
memory usage: 1.6+ MB


In [67]:
# exécuter la requête  suivante 
requete = """
select * from outcomes as o join street as s on o.`CrimeID` = s.`CrimeID` 
where o.`Reportedby` != s.`Reportedby`  or o.`Fallswithin` != s.`Fallswithin`
or o.Longitude != s.Longitude or s.Latitude != o.Latitude 
or o.Location != s.Location or o.`LSOAcode` != s.`LSOAcode` 
or o.`LSOAname` != s.`LSOAname`
"""

connection = sqlite3.connect(db_path)
df = pd.read_sql_query(requete, connection)
print(df)
connection.close()
